In [1]:
# Imports
import pandas as pd
import numpy as np
from scipy.stats import norm, uniform
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import trange
from shared.config_plt import config_plt, get_fig_ax, save_my_fig, colors
from shared.utils import print_se_parentheses, rank
from shared.univariate import DiscretizedDist, PriorAnalyzer, RankSumAnalyzer, RoundedAnalyzer, NeighborhoodAnalyzer, BRIAsympAnalyzer, DiffMeansAnalyzer, LIBDiffMeansAnalyzer
theta_rng = np.random.default_rng(seed=1)
data_rng = np.random.default_rng(seed=2)
config_plt()

In [2]:
# Setup
theta_abs_lim = 5.
theta_min = -theta_abs_lim
theta_max = theta_abs_lim
theta_inc = 0.001
theta_vals = np.arange(theta_min, theta_max+0.00001, theta_inc)
n_theta_vals = theta_vals.size
alpha = 0.05
n_each_vals = np.array([10, 40, 200, 1000])
n_sample_sizes = n_each_vals.size

prior_dist = DiscretizedDist(norm(loc=0., scale=1.), theta_vals, theta_rng)
methods = [
    PriorAnalyzer("Prior", alpha, prior_dist),
    DiffMeansAnalyzer("DIM", alpha, prior_dist),
    LIBDiffMeansAnalyzer("LIB", alpha, prior_dist),
    BRIAsympAnalyzer("BRI-A", alpha, prior_dist),
]
method_names = [m.name for m in methods]
n_methods = len(methods)

In [3]:
# Simulation
n_reps = 10000
posterior_means = np.zeros((n_reps, n_sample_sizes, n_methods))
lower_bounds = np.zeros((n_reps, n_sample_sizes, n_methods))
upper_bounds = np.zeros((n_reps, n_sample_sizes, n_methods))
nominal_coverage_rates = np.zeros((n_reps, n_sample_sizes, n_methods))
true_thetas = np.zeros((n_reps, n_sample_sizes))

def sample_a(n_arange, n_each):
    a_idx = data_rng.choice(n_arange, n_each, replace=False)
    a_sample = np.isin(n_arange, a_idx)
    return a_sample

# Simulation
for i in trange(n_reps):
    for k, n_each in enumerate(n_each_vals):
        # Generate data
        n = 2*n_each
        n_arange = np.arange(n)
        a_init = sample_a(n_arange, n_each)
        not_a_init = ~a_init
        y0_init = data_rng.normal(size=n) + data_rng.gamma(4., scale=0.25, size=n)
        y1_init = y0_init + 1. + data_rng.normal(size=n)
        ya_init = a_init*y1_init + not_a_init*y0_init
        true_theta_i = prior_dist.rvs()
        y0 = ya_init - a_init*true_theta_i
        y1 = y0 + true_theta_i
        true_thetas[i, k] = true_theta_i
        a = sample_a(n_arange, n_each)
        not_a = ~a
        y = a*y1 + not_a*y0

        for j, method in enumerate(methods):
            # Get posterior probs
            pm, lb, ub, cr = method.analyze(y, a, theta_vals)
            posterior_means[i, k, j] = pm
            lower_bounds[i, k, j] = lb
            upper_bounds[i, k, j] = ub
            nominal_coverage_rates[i, k, j] = cr

100%|██████████████████████████████████████████████████████████████████| 10000/10000 [1:36:11<00:00,  1.73it/s]


In [7]:
# Get mean and se
def get_mean_and_se(x, digits=3, stars=False, alpha=0.05):
    mean = x.mean(axis=0)
    se = x.std(axis=0) / np.sqrt(x.shape[0])
    mean_se = print_se_parentheses(mean, se, digits=digits, stars=stars, alpha=alpha)
    return mean_se

# Process results
errors = (posterior_means.T - true_thetas.T).T
errors2 = errors**2
ci_covered = ((lower_bounds.T <= true_thetas.T) & (true_thetas.T <= upper_bounds.T)).T
ci_length = upper_bounds - lower_bounds

arrs_to_process = [errors, errors2, ci_covered, nominal_coverage_rates, ci_length]
arrs_names_series = pd.Series(["Bias", "MSE", "CI Coverage", "CI Nominal Level", "CI Length"], name="Metrics")
method_names_series = pd.Series(method_names, name="Methods")

results_df = pd.DataFrame(
    np.vstack([
        np.stack([get_mean_and_se(arr[:, i], digits=4) for i in range(n_sample_sizes)])
        for arr in arrs_to_process]),
    index=pd.MultiIndex.from_product([arrs_names_series, n_each_vals.astype(str)]),
    columns=method_names_series)
results_df

Methods                           Prior               DIM              LIB  \
Metrics                                                                      
Bias             10     0.0062 (0.0099)  -0.0041 (0.0067)  0.0406 (0.0058)   
                 40    -0.0143 (0.0100)  -0.0023 (0.0033)  0.0155 (0.0032)   
                 200    0.0130 (0.0100)  -0.0004 (0.0015)  0.0047 (0.0015)   
                 1000   0.0068 (0.0100)   0.0006 (0.0007)  0.0016 (0.0007)   
MSE              10     0.9810 (0.0143)   0.4520 (0.0066)  0.3403 (0.0053)   
                 40     0.9983 (0.0140)   0.1104 (0.0016)  0.1040 (0.0016)   
                 200    0.9963 (0.0142)   0.0220 (0.0003)  0.0218 (0.0003)   
                 1000   1.0030 (0.0139)   0.0045 (0.0001)  0.0044 (0.0001)   
CI Coverage      10     0.9511 (0.0022)   0.9204 (0.0027)  0.9245 (0.0026)   
                 40     0.9539 (0.0021)   0.9452 (0.0023)  0.9418 (0.0023)   
                 200    0.9507 (0.0022)   0.9540 (0.0021)  0.9552 (0.0021)   
                 1000   0.9506 (0.0022)   0.9502 (0.0022)  0.9530 (0.0021)   
CI Nominal Level 10     0.9501 (0.0000)   0.9500 (0.0000)  0.9502 (0.0000)   
                 40     0.9501 (0.0000)   0.9500 (0.0000)  0.9504 (0.0000)   
                 200    0.9501 (0.0000)   0.9500 (0.0000)  0.9508 (0.0000)   
                 1000   0.9501 (0.0000)   0.9500 (0.0000)  0.9518 (0.0000)   
CI Length        10     3.9210 (0.0000)   2.4535 (0.0052)  2.0534 (0.0030)   
                 40     3.9210 (0.0000)   1.2830 (0.0019)  1.2168 (0.0016)   
                 200    3.9210 (0.0000)   0.5828 (0.0008)  0.5771 (0.0007)   
                 1000   3.9210 (0.0000)   0.2611 (0.0003)  0.2615 (0.0003)   

Methods                          BRI-A  
Metrics                                 
Bias             10    0.0440 (0.0059)  
                 40    0.0166 (0.0032)  
                 200   0.0047 (0.0015)  
                 1000  0.0016 (0.0007)  
MSE              10    0.3523 (0.0058)  
                 40    0.1048 (0.0017)  
                 200   0.0218 (0.0003)  
                 1000  0.0044 (0.0001)  
CI Coverage      10    0.9548 (0.0021)  
                 40    0.9520 (0.0021)  
                 200   0.9573 (0.0020)  
                 1000  0.9536 (0.0021)  
CI Nominal Level 10    0.9502 (0.0000)  
                 40    0.9504 (0.0000)  
                 200   0.9508 (0.0000)  
                 1000  0.9517 (0.0000)  
CI Length        10    2.3494 (0.0036)  
                 40    1.2675 (0.0017)  
                 200   0.5820 (0.0007)  
                 1000  0.2619 (0.0003)

In [8]:
print(results_df.to_latex())

\begin{tabular}{llllll}
\toprule
 & Methods & Prior & DIM & LIB & BRI-A \\
Metrics &  &  &  &  &  \\
\midrule
\multirow[t]{4}{*}{Bias} & 10 & 0.0062 (0.0099) & -0.0041 (0.0067) & 0.0406 (0.0058) & 0.0440 (0.0059) \\
 & 40 & -0.0143 (0.0100) & -0.0023 (0.0033) & 0.0155 (0.0032) & 0.0166 (0.0032) \\
 & 200 & 0.0130 (0.0100) & -0.0004 (0.0015) & 0.0047 (0.0015) & 0.0047 (0.0015) \\
 & 1000 & 0.0068 (0.0100) & 0.0006 (0.0007) & 0.0016 (0.0007) & 0.0016 (0.0007) \\
\cline{1-6}
\multirow[t]{4}{*}{MSE} & 10 & 0.9810 (0.0143) & 0.4520 (0.0066) & 0.3403 (0.0053) & 0.3523 (0.0058) \\
 & 40 & 0.9983 (0.0140) & 0.1104 (0.0016) & 0.1040 (0.0016) & 0.1048 (0.0017) \\
 & 200 & 0.9963 (0.0142) & 0.0220 (0.0003) & 0.0218 (0.0003) & 0.0218 (0.0003) \\
 & 1000 & 1.0030 (0.0139) & 0.0045 (0.0001) & 0.0044 (0.0001) & 0.0044 (0.0001) \\
\cline{1-6}
\multirow[t]{4}{*}{CI Coverage} & 10 & 0.9511 (0.0022) & 0.9204 (0.0027) & 0.9245 (0.0026) & 0.9548 (0.0021) \\
 & 40 & 0.9539 (0.0021) & 0.9452 (0.0023) & 0.941